In [1]:
import JupyterNotebooksLib as slicernb
import slicer
import json
import glob
import os
import vtk
import numpy as np
import ScreenCapture
import copy
import shutil
import ctk
from datetime import date

today = date.today()
slicernb.AppWindow.setWindowSize(scale=3.0)
slicernb.showSliceViewAnnotations(False)

In [2]:
def lineNode_(volnode,point1,point2):
    volumeIjkToRas = vtk.vtkMatrix4x4()
    volume.GetIJKToRASMatrix(volumeIjkToRas)
    
    point_VolumeRas = [0, 0, 0, 1]
    point_VolumeRas2 = [0, 0, 0, 1]
    volumeIjkToRas.MultiplyPoint(np.append(point1,1.0), point_VolumeRas)
    volumeIjkToRas.MultiplyPoint(np.append(point2,1.0), point_VolumeRas2)
    transformVolumeRasToRas = vtk.vtkGeneralTransform()
    slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(volnode.GetParentTransformNode(), None, transformVolumeRasToRas)
    slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(segnode.GetParentTransformNode(), None, transformVolumeRasToRas)
    
    point_Ras = transformVolumeRasToRas.TransformPoint(point_VolumeRas[0:3])
    point_Ras2 = transformVolumeRasToRas.TransformPoint(point_VolumeRas2[0:3])

    lineNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsLineNode")
    lineNode.AddControlPoint((point_Ras[0], point_Ras[1], point_Ras[2]), "max")
    lineNode.AddControlPoint((point_Ras2[0], point_Ras2[1], point_Ras2[2]), "max")

    return lineNode

def getpointlist_(volnode,point1,point2,markupPoint):
    volumeIjkToRas = vtk.vtkMatrix4x4()
    volume.GetIJKToRASMatrix(volumeIjkToRas)
    
    point_VolumeRas = [0, 0, 0, 1]
    point_VolumeRas2 = [0, 0, 0, 1]
    point_volmarkup = [0, 0, 0, 1]
    volumeIjkToRas.MultiplyPoint(np.append(point1,1.0), point_VolumeRas)
    volumeIjkToRas.MultiplyPoint(np.append(point2,1.0), point_VolumeRas2)
    volumeIjkToRas.MultiplyPoint(np.append(markupPoint,1.0), point_volmarkup)
    
    transformVolumeRasToRas = vtk.vtkGeneralTransform()
    slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(volnode.GetParentTransformNode(), None, transformVolumeRasToRas)
    slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(segnode.GetParentTransformNode(), None, transformVolumeRasToRas)
    
    point_Ras = transformVolumeRasToRas.TransformPoint(point_VolumeRas[0:3])
    point_Ras2 = transformVolumeRasToRas.TransformPoint(point_VolumeRas2[0:3])
    point_markup = transformVolumeRasToRas.TransformPoint(point_volmarkup[0:3])
    
    markupNode = slicer.mrmlScene.AddNode(slicer.vtkMRMLMarkupsFiducialNode())
    markupNode.AddControlPoint((point_Ras[0], point_Ras[1], point_Ras[2]), "min")
    markupNode.AddControlPoint((point_Ras2[0], point_Ras2[1], point_Ras2[2]), "max")
    markupNode.AddControlPoint((point_markup[0], point_markup[1], point_markup[2]), "markup")
    return markupNode

In [3]:
def CaptureMRIwithmaxIAM(volnode, json_object):
    croppedVolume, cropVolumeParameterNode, center_tumor = CropMRIBoundingBoxExtended(volnode,json_object)
    markupPoint = json_object['markupPoint']   #extra markup point from IJK to get the plane in RAS format. 
    pointListNode = getpointlist_(volnode,json_object["maxIAMDiameterCoordinates"][0],json_object["maxIAMDiameterCoordinates"][1],markupPoint)
    # position_axial = pointListNode.GetNthControlPointPosition(2)
    nOfFiduciallPoints = pointListNode.GetNumberOfFiducials()
    points = np.zeros([3,nOfFiduciallPoints])
    for i in range(0, nOfFiduciallPoints):
        pointListNode.GetNthFiducialPosition(i, points[:,i])
    # Compute plane position and normal
    planePosition = points.mean(axis=1)
    sliceNode = slicer.app.layoutManager().sliceWidget("Red").mrmlSliceNode()
    sliceNode.JumpSliceByOffsetting(planePosition[0],planePosition[1],planePosition[2])
    slicer.mrmlScene.RemoveNode(pointListNode)

    cap = ScreenCapture.ScreenCaptureLogic()
    cap.captureImageFromView(view, os.path.join(path_new,"%s_%s_MRI.png"%(json_object['ID'],json_object['Time_point'])))
#     slicer.mrmlScene.RemoveNode(volnode)
    slicer.mrmlScene.RemoveNode(cropVolumeParameterNode)
    slicer.mrmlScene.RemoveNode(croppedVolume)
    

def CropMRIBoundingBoxExtended(volnode,json_object):
    viewNode = slicer.mrmlScene.GetSingletonNode("1", "vtkMRMLViewNode")
    cameraNode = slicer.modules.cameras.logic().GetViewActiveCameraNode(viewNode)
    #get the bounding box center for the camera to focus
    l_bb = lineNode_(volnode,json_object["boundary_bb"][0],json_object["boundary_bb"][1])
    roinode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
    center_tumor = (l_bb.GetMarkupPointVector(0,0)[0]+l_bb.GetMarkupPointVector(1,0)[0])/2,(l_bb.GetMarkupPointVector(0,0)[1]+l_bb.GetMarkupPointVector(1,0)[1])/2,(l_bb.GetMarkupPointVector(0,0)[2]+l_bb.GetMarkupPointVector(1,0)[2])/2
    slicer.mrmlScene.RemoveNode(roinode)
    slicer.mrmlScene.RemoveNode(l_bb)
    
    
    # Get the bounding box (extended) and crop the region 

    l_bb = lineNode_(volnode,json_object["boundary_bb_extended"][0],json_object["boundary_bb_extended"][1])

    roinode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
    size_bb = l_bb.GetMarkupPointVector(0,0)[0]-l_bb.GetMarkupPointVector(1,0)[0],l_bb.GetMarkupPointVector(0,0)[1]-l_bb.GetMarkupPointVector(1,0)[1],l_bb.GetMarkupPointVector(0,0)[2]-l_bb.GetMarkupPointVector(1,0)[2]
    center_tumor = (l_bb.GetMarkupPointVector(0,0)[0]+l_bb.GetMarkupPointVector(1,0)[0])/2,(l_bb.GetMarkupPointVector(0,0)[1]+l_bb.GetMarkupPointVector(1,0)[1])/2,(l_bb.GetMarkupPointVector(0,0)[2]+l_bb.GetMarkupPointVector(1,0)[2])/2
    roinode.SetSize(size_bb)
    roinode.SetCenter(center_tumor)

    cropVolumeLogic = slicer.modules.cropvolume.logic()
    cropVolumeParameterNode = slicer.vtkMRMLCropVolumeParametersNode()
    cropVolumeParameterNode.SetROINodeID(roinode.GetID())
    cropVolumeParameterNode.SetInputVolumeNodeID(volnode.GetID())
    cropVolumeParameterNode.SetVoxelBased(True)
    cropVolumeLogic.Apply(cropVolumeParameterNode)
    
    croppedVolume = slicer.mrmlScene.GetNodeByID(cropVolumeParameterNode.GetOutputVolumeNodeID())
    slicer.mrmlScene.RemoveNode(roinode)
    slicer.mrmlScene.RemoveNode(volnode)
    slicer.mrmlScene.RemoveNode(l_bb)

    set_red_view(Visibility=True)

    return croppedVolume, cropVolumeParameterNode, center_tumor


In [4]:
def init_view(T2_im_path,seg_path,json_object):   
    markupPoint = json_object['markupPoint']   #extra markup point from IJK to get the plane in RAS format. 
    
    
    volume = slicer.util.loadVolume(T2_im_path)
    seg = slicer.util.loadSegmentation(seg_path)
    segnode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
    volnode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLVolumeNode")
    
    
    #Setting up the slicer view 
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    segmentNames = ["Extra", "Intra","Whole"]
    segmentIds = vtk.vtkStringArray()
    for segmentName in segmentNames:
      segmentId = segnode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
      segmentIds.InsertNextValue(segmentId)
    slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(segnode, segmentIds, labelmapVolumeNode, volnode)
    
    slicer.mrmlScene.RemoveNode(labelmapVolumeNode)
    
    view = slicer.app.layoutManager().threeDWidget(0).threeDView()
    view.mrmlViewNode().SetBackgroundColor(0,0,0)
    view.mrmlViewNode().SetBackgroundColor2(0,0,0)
    view.mrmlViewNode().SetBoxVisible(False)
    view.mrmlViewNode().SetAxisLabelsVisible(False)

    renderWindow = view.renderWindow()
    renderWindow.SetAlphaBitPlanes(1)

    segnode.GetDisplayNode().SetSegmentOpacity2DFill("Segment_1", 0.4)
    segnode.GetSegmentation().SetConversionParameter('Smoothing factor','0') # disable 3D smoothing
    segnode.CreateClosedSurfaceRepresentation() # show segmentation in 3D
    segnode.GetDisplayNode().SetOpacity3D(0.7)
    segnode.GetDisplayNode().SetVisibility(True)
    return volnode,segnode,volume, renderWindow

In [5]:
def set_red_view(Visibility=True):
    #set the screen shot capturing view ot the axial (Red) view
    red = slicer.util.getNode('vtkMRMLSliceNodeRed')
    # Capture RGBA image
    slicernb.ViewDisplay("FourUp")
    
    sliceViewName = "Red"
    # Set view background to white
    view = slicer.app.layoutManager().sliceWidget(sliceViewName).sliceView()
    view.forceRender()

    layoutManager = slicer.app.layoutManager()
    redWidget = layoutManager.sliceWidget('Red')
    redLogic = redWidget.sliceLogic()
    redLogic.FitSliceToAll() 

    # Capture RGBA image
    red.SetSliceVisible(Visibility)

    slicernb.ViewDisplay("FourUp")
    return view


In [6]:
# Clear scene
slicer.mrmlScene.Clear(False)

In [7]:

list_json = sorted(glob.glob('../SampleCase/features_json_2024-01-28/*.json'))
subjects_path  = '/home/nw22/Documents/MDM/AutomatedReportGeneration/SampleCase/MRI/'
for file in list_json:
    with open(file, 'r') as openfile:
        json_object = json.load(openfile)
    id_name = json_object['ID']
    path_new = os.path.join('../SampleCase/images_%s/'%str(today))
    isExist = os.path.exists(path_new)
    if not isExist:
      os.makedirs(path_new)
    T2_im_path = os.path.join(subjects_path,json_object['ID'],json_object['Time_point'],'%s.nii.gz'%(json_object['MRI_modality']))
    seg_path = os.path.join(subjects_path,json_object['ID'],json_object['Time_point'],'%s_seg.nii.gz'%(json_object['MRI_modality']))
    volnode,segnode,volume, renderWindow = init_view(T2_im_path,seg_path,json_object)
    view = set_red_view(Visibility=True)
    CaptureMRIwithmaxIAM(volnode, json_object)
    slicer.mrmlScene.Clear()
